In [81]:
#Prepare a Jupyter notebook that contains an analysis of a classification problem using Logistic Regression. 
#Describe the dataset including either an explicit data dictionary (in the case of a smaller dataset) or a link to an original data dictionary

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
adm = pd.read_csv('admissions.csv')
adm

#Explicitly frame your classification problem; what are you measuring and how are you going to understand the quality of your model?

#Describe any data cleaning and feature manipulation prior to model implementation
adm = adm.dropna()
adm.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 399
Data columns (total 4 columns):
admit       397 non-null int64
gre         397 non-null float64
gpa         397 non-null float64
prestige    397 non-null float64
dtypes: float64(3), int64(1)
memory usage: 15.5 KB


In [82]:
X=adm.loc[:,'gre':'prestige']
y=adm['admit']

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y) 



In [84]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [85]:
from sklearn.metrics import mean_squared_error
y_pred = log_reg.predict(X_test)
log_mse = mean_squared_error(y_test, y_pred)
log_mse

0.32000000000000001

In [89]:
# cross-validation in training
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LogisticRegression(),X_train,y_train,cv=5)
print(np.sqrt(scores))

[ 0.80977633  0.84372057  0.88298415  0.85370578  0.85370578]


In [91]:
#Describe any parameter tuning or normalization that took place
#Fine-Tune Your Model
#Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
grid_search = GridSearchCV(log_reg, param_grid, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [92]:
#the best Parameter
grid_search.best_params_


{'C': 1000}

In [93]:
# the best esitmator
grid_search.best_estimator_


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [95]:
#Measuring Accuracy Using Cross-Validation
# not very accurate based on accuracy score
from sklearn.model_selection import cross_val_score
cross_val_score(log_reg, X_train, y_train, cv=5, scoring="accuracy")

array([ 0.6557377 ,  0.71186441,  0.77966102,  0.72881356,  0.72881356])

In [96]:
#Confusion Matrix
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(log_reg, X_train, y_train, cv=5)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)

array([[192,  14],
       [ 69,  22]])

In [98]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train, y_train_pred)

0.61111111111111116

In [99]:
recall_score(y_train, y_train_pred)

0.24175824175824176

In [102]:
#F1 score
from sklearn.metrics import f1_score
f1_score = f1_score(y_train, y_train_pred)

In [107]:
#Plot and interpret the following:
#Precision and Recall curves together (thresholds vs. probability)
#Precision vs. Recall
#ROC curve

from sklearn.metrics import precision_recall_curve
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])

precisions, recalls, thresholds = precision_recall_curve(y_train, scores)

ValueError: Found input variables with inconsistent numbers of samples: [297, 5]